In [38]:
!pip install accelerate
import accelerate
!pip install sentencepiece
import sentencepiece
!pip install transformers
import transformers
!pip install torch
import torch

NotImplementedError: ignored

In [39]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("stabilityai/StableBeluga-7B", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "stabilityai/StableBeluga-7B",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
system_prompt = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.\n\n"

message = "Write me a poem please"
prompt = f"{system_prompt}### User: {message}\n\n### Assistant:\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(
    **inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
system_prompt = """### System:\nInstruction:

**Prompt:** Extract filters from the query and return the result in a structured JSON format. The input query will consist of two parts: a primary query and a filter. The filter will typically be in the format "from [start_date] to [end_date]" or some variation of it. Your task is to:

1. Identify and extract the primary query, which represents the main search topic and may include important keywords like rake_nltk model.
2. Extract the start date and end date from the filter, which can be in the form of:
   - A full date (YYYY-MM-DD)
   - A month and year (YYYY-Month), where:
     - The start date should be the first day of the month (YYYY-MM-01).
     - The end date should be the last day of the month (YYYY-MM-last day).
   - A year (YYYY), where:
     - The start date should be the first day of the year (YYYY-01-01).
     - The end date should be the last day of the year (YYYY-12-31).
3. Extract the start amount range and end amount range from the filter, which can include the following:
   - Extract the start amount range and end amount range
   - "under" specifying a start range set to 0 and ending at a specified amount.
   - "above" specifying a start range from a specified amount and end range set to 9999999999.
4. Extract a single or list of UNSPSC Codes, each consisting of 8 digits, from the filter, if present.
5. Determine the date type based on the following context keywords before the date:
   - "Tender Closing Date"
   - "Start Date"
   - "Initial Expiry Date"
   - "Expiry Date"
   - "Tender Open Date"
   - "Tender Close Date"
   - If any of these keywords is mentioned before a date, set the date type accordingly.
   - By default, the date type is "Awarded Date."
6. Determine the cost range type based on the following context keywords before the cost range:
   - "Original Contract Value"
   - "Supplier Contract Price"
   - "Variation Amount"
   - "Contract Expenditure"
   - If any of these keywords is mentioned before a cost range, set the cost range type accordingly.
   - By default, the cost range type is "Revised Contract Value."
7. Extract the "Region" from the from the filter, if present.

Please make sure the JSON format for the response adheres to the following structure:

```
{
  "query": "Primary Query",
  "start_date": Date (YYYY-MM-DD),
  "end_date": Date (YYYY-MM-DD),
  "unspsc_code" : UNSPSC_Code,
  "type_of_work" : Type_of_Work,
  "start_range": Integer ($),
  "end_range": Integer ($),
  "date_type": Date_Type,
  "cost_range_type": Cost_Range_Type,
  "region": "Region_Name",
  "reference_number": "Reference_Number",
}
```

\n\n"""
message = "contracts having CCTV installation for Expiry Date as September 2022 Variation Amount under $500 with UNSPSC Code 80172000, 80101513, 41102614 from South West"
prompt = f"{system_prompt}### User: {message}\n\n### Assistant:\n"
prompt_tokens = tokenizer(prompt, return_tensors="pt").to("cuda")["input_ids"]
start_index = prompt_tokens.shape[-1]

output = model.generate(
    prompt_tokens, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256
)

generation_output = output[0][start_index:]
generation_text = tokenizer.decode(generation_output, skip_special_tokens=True)
print(generation_text)

In [ ]:
import json

data = json.loads(generation_text)

print(data)